## Profiling with PyTorch Module

**Note**

Profiler supports multi-threaded models. Profiler runs in the same thread as the operation but it will also profile child operators that might be running in another thread.

In case there are concurrent profilers running then each profiler will be scoped to its thread to prevent the mixing of results.

In [1]:
import torch
import torchvision.models as models
from torch.profiler import profile, record_function, ProfilerActivity

In [2]:
# Initialize resnet18 & model inputs
model = models.resnet18()
inputs = torch.randn(5, 3, 244, 244)

**Execution Time Analysis**

`activities` - The list of activities to profile

- `ProfilerActivity.CPU` - PyTorch operators, TorchScript functions & user-defined code labels
- `ProfilerActivity.CUDA` - on device cuda kernels

`record_shapes` - whether to record shapes of the operator inputs

`profile_memory` - whether to report amount of memory consumed by the model's Tensors

`use_cuda` - whether to measure execution time of CUDA kernels

In [3]:
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
  with record_function("Model Inference"):
    model(inputs)

In [4]:
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                  Model Inference         2.15%      41.016ms       100.00%        1.910s        1.910s             1  
                     aten::conv2d         0.12%       2.351ms        69.75%        1.332s      66.600ms            20  
                aten::convolution         0.17%       3.281ms        69.63%        1.330s      66.482ms            20  
               aten::_convolution         0.04%     784.000us        69.45%        1.326s      66.318ms            20  
         aten::mkldnn_convolution        68.25%        1.303s        69.41%        1.326s      66.279ms            20  
                 aten::batch_norm       

**Notes**

The `self CPU` shows the time for the operator. An operator can call multiple operators. `self cpu time` excludes the time spent in children operator calls.

`CPU Total` indicates the total time

In [5]:
# Finer granuler results & include operator shapes
print(prof.key_averages(group_by_input_shape=True).table(sort_by="cpu_time_total", row_limit=10))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls                                                                      Input Shapes  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  
                  Model Inference         2.15%      41.016ms       100.00%        1.910s        1.910s             1                                                                                []  
                     aten::conv2d         0.00%      73.000us        17.28%     329.987ms      82.497ms             4                             [[5, 64, 61, 61], [64, 64, 3, 3], [], [], [], 

### CPU & CUDA Analysis

**NOTE**

The first profiling of CUDA will bring an extra overhead

For Example, running the following cell for the first time results
- Self CPU time total: 806.113ms
- Self CUDA time total: 10.680ms

Then running it again
- Self CPU time total: 11.737ms
- Self CUDA time total: 10.707ms


In [8]:
model = models.resnet18().cuda()
inputs = torch.randn(5, 3, 244, 244).cuda()

with profile(
    activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
    record_shapes=True,
) as prof:
  with record_function("model_inference"):
    model(inputs)

print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                        model_inference        20.47%       2.394ms        80.49%       9.413ms       9.413ms       0.000us         0.00%      10.484ms      10.484ms             1  
                                           aten::conv2d         0.86%     100.000us        24.35%       2.848ms     142.400us       0.000us         0.00%       8.347ms     417.350us            20  
         

### Profiling memory consumption

`self` - corresponds to the memory allocated (released) by the operator excluding all the children calls to the other operators

In [2]:
model = models.resnet18()
inputs = torch.randn(5, 3, 224, 224)

with profile(activities=[ProfilerActivity.CPU],
        profile_memory=True, record_shapes=True) as prof:
    model(inputs)

print(prof.key_averages().table(sort_by="self_cpu_memory_usage", row_limit=10))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                      aten::empty         0.17%     692.000us         0.17%     692.000us       3.460us      89.10 Mb      89.10 Mb           200  
    aten::max_pool2d_with_indices        20.93%      87.142ms        20.93%      87.142ms      87.142ms      11.48 Mb      11.48 Mb             1  
                 aten::empty_like         0.03%     112.000us         0.05%     206.000us      10.300us      47.37 Mb       5.74 Mb            20  
                      aten::addmm         0.13%     540.000us         0.14%     567.000us     567.000us      19.

In [3]:
"""
In the below table, aten::conv2d & aten::convolution etc
- Are displaying memory usage (CPU Mem)

Whereas they have no `Self CPU Mem`. It seems that these are the result of child calls to the operator
which is captured in the CPU Mem
"""
print(prof.key_averages().table(sort_by="cpu_memory_usage", row_limit=10))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                      aten::empty         0.17%     692.000us         0.17%     692.000us       3.460us      89.10 Mb      89.10 Mb           200  
                 aten::batch_norm         0.03%     105.000us        12.85%      53.487ms       2.674ms      47.41 Mb           0 b            20  
     aten::_batch_norm_impl_index         0.06%     262.000us        12.82%      53.382ms       2.669ms      47.41 Mb           0 b            20  
          aten::native_batch_norm        12.65%      52.645ms        12.75%      53.086ms       2.654ms      47.

In [4]:
# Tracing functionality [ can be opened with chrome trace viewer `chrome:://tracing` ]
model = models.resnet18().cuda()
inputs = torch.randn(5, 3, 224, 224).cuda()

with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA]) as prof:
    model(inputs)

prof.export_chrome_trace("trace.json")

In [5]:
# Examining stack trace in profiler (for some reason `Source Location` is not present)
with profile(
    activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
    with_stack=True,
) as prof:
    model(inputs)

# Print aggregated stats
print(prof.key_averages(group_by_stack_n=5).table(sort_by="self_cuda_time_total", row_limit=2))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                aten::cudnn_convolution        39.77%       4.067ms        44.41%       4.542ms     227.100us       7.859ms        81.67%       7.859ms     392.950us            20  
cudnn_infer_volta_scudnn_winograd_128x128_ldg1_ldg4_...         0.00%       0.000us         0.00%       0.000us       0.000us       5.571ms        57.89%       5.571ms     428.538us            13  
---------

### Using profiler to analyze long running jobs

Parameter `skip_first` tells profiler that it should ignore the first 10 steps (default value of `skip_first` is zero);

After the first skip_first steps, profiler starts executing profiler cycles;

Each cycle consists of three phases:

- idling (wait=5 steps), during this phase profiler is not active;

- warming up (warmup=1 steps), during this phase profiler starts tracing, but the results are discarded; this phase is used to discard the samples obtained by the profiler at the beginning of the trace since they are usually skewed by an extra overhead;

- active tracing (active=3 steps), during this phase profiler traces and records data;

An optional repeat parameter specifies an upper bound on the number of cycles. By default (zero value), profiler will execute cycles as long as the job runs.

Whenever a trace is available the profiler calls the user defined `trace handler function`. The profiler pass a reference of itself to this function which can be used for creating tables etc.



In [6]:
# Using profiler to analyze long running jobs
from torch.profiler import schedule

# generate a schedule
my_schedule = schedule(
    skip_first=10,
    wait=5,
    warmup=1,
    active=3,
    repeat=2)

In [7]:
def trace_handler(p):
    output = p.key_averages().table(sort_by="self_cuda_time_total", row_limit=10)
    print(output)
    p.export_chrome_trace("trace_" + str(p.step_num) + ".json")

with profile(
    activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
    schedule=torch.profiler.schedule(
        wait=1,
        warmup=1,
        active=2),
    on_trace_ready=trace_handler
) as p:
    for idx in range(8):
        model(inputs)
        p.step()

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                aten::cudnn_convolution        10.10%       2.991ms        14.75%       4.366ms     109.150us      15.738ms        81.60%      15.738ms     393.450us            40  
cudnn_infer_volta_scudnn_winograd_128x128_ldg1_ldg4_...         0.00%       0.000us         0.00%       0.000us       0.000us      11.145ms        57.79%      11.145ms     428.654us            26  
         